In [ ]:
from osgeo import gdal
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
import json
from osgeo import ogr, osr
import numpy as np
import csv
import geopandas as gpd

In [ ]:
path = "denhaag_grouped_roof.csv"
df = pd.read_csv(path)
df.head()


In [ ]:
roofSurfaces = {}
for i,row in df.iterrows():
    if "MULTIPOLYGON" in row["geometry"]:
        # multipolygons = eval(row["geometry"])
        roofSurfaces[row["pri_key"]] = row["geometry"]
    else:
        print("bad format")

In [ ]:
driver = ogr.GetDriverByName('GeoJSON')
ds = driver.CreateDataSource('denhaag_roof.geojson')
if ds is None:
    print("no driver")

srs = osr.SpatialReference()
srs.ImportFromEPSG(7415)  # Set the coordinate system (e.g., WGS84)
layer = ds.CreateLayer('output.geojson', srs, ogr.wkbMultiPolygon)




In [ ]:
field_defn1 = ogr.FieldDefn('Id', ogr.OFTString)

field_defn1.SetWidth(50)
field_defn2 = ogr.FieldDefn('Area', ogr.OFTReal)
field_defn3 = ogr.FieldDefn('MaxAngle', ogr.OFTReal)
field_defn4 = ogr.FieldDefn('MinAngle', ogr.OFTReal)
field_defn5 = ogr.FieldDefn('Num_polygon', ogr.OFTReal)
field_defn6 = ogr.FieldDefn('MaxAreaPropotion', ogr.OFTReal)
field_defn7 = ogr.FieldDefn('LargestPolygonAngle', ogr.OFTReal)
layer.CreateField(field_defn1)
layer.CreateField(field_defn2)
layer.CreateField(field_defn3)
layer.CreateField(field_defn4)
layer.CreateField(field_defn5)
layer.CreateField(field_defn6)
layer.CreateField(field_defn7)

In [ ]:
def ground_polygonAngles(polygon):
    centroid = polygon.Centroid()
    ring = polygon.GetGeometryRef(0)
    centroid = [centroid.GetX(),centroid.GetY(),centroid.GetZ()]
    z_coordinates = [ring.GetPoint(n)[2] for n in range(ring.GetPointCount())]
    centroid_z = sum(z_coordinates) / len(z_coordinates)
    centroid[2]=centroid_z

    points = [ring.GetPoint(n) for n in range(ring.GetPointCount())]
    normal_vector = np.cross(np.array(points[0]) - np.array(centroid), np.array(points[1]) - np.array(centroid))
    normal_vector /= np.linalg.norm(normal_vector)
    ground_normal = np.array([0, 0, -1])  # Assuming "down" is the ground
    angles = np.arccos(np.dot(ground_normal, normal_vector))
    angles= round(angles*180/np.pi,2)
    return angles

In [ ]:
ground_normal = np.array([0, 0, -1])
for id, layers in roofSurfaces.items():
    multi_polygon = ogr.CreateGeometryFromWkt(layers)
    angles = []
    areas = []
    if multi_polygon:  
        for i in range(0, multi_polygon.GetGeometryCount()):
            # polygon = ogr.Geometry(ogr.wkbPolygon)
            # for rings in polygons:
            #     ring = ogr.Geometry(ogr.wkbLinearRing)
            #     for coord in rings:
            #         ring.AddPoint(coord[0], coord[1], coord[2])
            #     ring.AddPoint(rings[0][0], rings[0][1],rings[0][2])
            #     polygon.AddGeometry(ring)
            # calculate the angle between polygon and ground
            # multi_polygon.AddGeometry(polygon)
            polygon = multi_polygon.GetGeometryRef(i)
            if polygon.IsValid():
                areas.append(polygon.Area())
                angle = ground_polygonAngles(polygon)
                angles.append(angle)
        if angles:
            max_angle = max(angles)
            min_angle = min(angles)
            indices = np.argmax(areas)
            largest_area_angle = angles[indices]
            feature = ogr.Feature(layer.GetLayerDefn())
            feature.SetGeometry(multi_polygon)
            polygon_area = multi_polygon.Area()
            num_polygon = multi_polygon.GetGeometryCount()
            feature.SetField('Area', polygon_area)
            feature.SetField('Id', id)
            feature.SetField('MaxAngle', max_angle)
            feature.SetField('MinAngle', min_angle)
            feature.SetField('Num_polygon', num_polygon)
            feature.SetField('LargestPolygonAngle', largest_area_angle)
            feature.SetField('MaxAreaPropotion', 100*(max(areas)/polygon_area))

            layer.CreateFeature(feature)
ds = None


In [ ]:
gdf = gpd.read_file("D:/Delft/Q5/roof/roof_geojeson/rotterdam_roof.geojson")

In [ ]:
gdf.head()

In [ ]:
gdf["geometry"]

In [ ]:
gdf.to_csv("rotterdam_roof.csv")